In [6]:
from playwright.async_api import async_playwright, Playwright
from rich import print
import asyncio

In [ ]:
from playwright.async_api import async_playwright
from collections import defaultdict
from urllib.parse import urljoin
import nest_asyncio
import json
import re
nest_asyncio.apply()
async def run(playwright):
    start_url = "https://aniagotuje.pl"
    browser = await playwright.chromium.launch(headless=True)
    page = await browser.new_page()
    await page.goto(start_url)

    recipecount=0
    recipemap = defaultdict(lambda: {"ingredient": [],"recipe_info":[]})
    recipemap_end = defaultdict(lambda: {"ingredient": [],"recipe_info":[]})

    while True:
        if recipecount<10:
            links = await page.locator('a[href^="/przepis/"]').all()
            if len(links)<20:
                break
            print(f"Znaleziono {len(links)} linków")
            
            linkcount=0    

            for link in links:
                if linkcount<30:
                    p = await browser.new_page(base_url=start_url)

                    href = await link.get_attribute('href')
                    
                    if "#" not in href:

                        if href not in recipemap.keys():
                            await p.goto(href)
                            print(href)
                            ingredients = await p.locator('span[class^="ingredient"]').all()
                            qty = await p.locator('span[class^="qty"]').all()
                            if len(qty)>0:
                                print("TEST")
                                for i in range(len(qty)):
                                    text_ingredients= await ingredients[i].text_content()
                                    text_qty= await qty[i].text_content()
                                    
                                    recipemap_end[href]["ingredient"].append(
                                        {
                                            "name":text_ingredients,
                                            "amount":text_qty
                                        }
                                    )
                                    if len(ingredients)> len(qty):
                                        recipemap_end[href]["ingredient"].append(ingredients[(len(qty)):])
                                for inf in recipe_info:
                                    inftext=await inf.text_content()
                                    inftext=inftext.replace("\t"," ")
                                    inftext=re.sub(r'\s{2,}',",",inftext)
                                    recipemap_end[href]["recipe_info"].append(inftext)
                            else:
                                for ing in ingredients:
                                    
                                    text = await ing.text_content()
                                    recipemap[href]["ingredient"].append(text)
                                    
                                recipe_info = await p.locator('p[class^="recipe-info"]').all()
                                
                                for inf in recipe_info:
                                    inftext=await inf.text_content()
                                    inftext=inftext.replace("\t"," ")
                                    inftext=re.sub(r'\s{2,}',",",inftext)
                                    recipemap[href]["recipe_info"].append(inftext)
                else:
                    break
                linkcount=linkcount+1 
        else:
            break
        recipecount=recipecount+1
        
        next_link = page.locator('a.page-link', has_text='»')
    
        if await next_link.count() > 0:
            href = await next_link.get_attribute('href')
            full_url = urljoin(start_url, href)
            if href:
                await page.goto(full_url)
                await page.wait_for_load_state('load')
            else:
                break
        else:
            break
    await browser.close()
    return recipemap,recipemap_end



async with async_playwright() as playwright:
    mappa,mappa_end = await run(playwright)

    with open("przepisy.json", "w", encoding="utf-8") as f:
        json.dump(mappa, f, ensure_ascii=False, indent=2)

Znaleziono 66 linków
/przepis/ogorki-kiszone
/przepis/ogorki-konserwowe
/przepis/jak-zrobic-ciasto-na-nalesniki
/przepis/jagodzianki
/przepis/salatka-z-ogorkow
/przepis/murzynek-z-wisniami
TEST
/przepis/gofry
/przepis/ogorki-kanapkowe
/przepis/placki-z-cukinii
/przepis/leczo
/przepis/leczo-z-cukinii
/przepis/domowe-ogorki-malosolne
/przepis/murzynek-z-wisniami
TEST
/przepis/murzynek-z-wisniami
TEST
/przepis/salatka-dakos
TEST
/przepis/salatka-dakos
TEST
/przepis/grzanki-na-slodko
TEST
/przepis/grzanki-na-slodko
TEST
/przepis/ciasta-z-malinami
/przepis/ciasta-z-malinami
/przepis/pistacjowy-sernik-baskijski
TEST
/przepis/pistacjowy-sernik-baskijski
TEST
Znaleziono 59 linków
/przepis/curry-z-kalafiora
TEST
/przepis/curry-z-kalafiora
TEST
/przepis/ciasto-czekoladowe-z-cukinii
TEST
/przepis/ciasto-czekoladowe-z-cukinii
TEST
/przepis/nalewka-z-czerwonej-porzeczki
TEST
/przepis/nalewka-z-czerwonej-porzeczki
TEST
/przepis/sernik-z-jagodami
TEST
/przepis/sernik-z-jagodami
TEST
/przepis/grzanki-

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


/przepis/muffinki-z-truskawkami


Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


TEST
/przepis/gofry-z-cukinia
TEST
/przepis/gofry-z-cukinia
TEST
/przepis/salatka-cezar-ze-szparagami
TEST
/przepis/salatka-cezar-ze-szparagami
TEST
/przepis/sernik-na-zimno-z-truskawkami
TEST
/przepis/sernik-na-zimno-z-truskawkami
TEST
/przepis/sernik-na-zimno-czekoladowy
TEST
/przepis/sernik-na-zimno-czekoladowy
TEST
/przepis/karmelizowane-tosty-francuskie-z-truskawkami
TEST
/przepis/karmelizowane-tosty-francuskie-z-truskawkami
TEST
/przepis/chlodnik-z-buraczkow
TEST
/przepis/chlodnik-z-buraczkow
TEST
Znaleziono 59 linków
/przepis/zupa-szparagowa-z-feta
TEST
/przepis/zupa-szparagowa-z-feta
TEST
/przepis/kasza-peczak-ze-szparagami
TEST
/przepis/kasza-peczak-ze-szparagami
TEST
/przepis/kopiec-shreka
TEST
/przepis/kopiec-shreka
TEST
/przepis/zupa-z-lososia
TEST
/przepis/zupa-z-lososia
TEST
/przepis/placki-z-rabarbarem
TEST
/przepis/placki-z-rabarbarem
TEST
/przepis/carbonara-ze-szparagami
TEST
/przepis/carbonara-ze-szparagami
TEST
Znaleziono 51 linków
/przepis/salatka-z-jajkami
TEST
/pr

In [23]:
print(mappa_end)

defaultdict(<function run.<locals>.<lambda> at 0x0000022E6B148A40>, {'/przepis/murzynek-z-wisniami': {'ingredient': [{'name': <Locator frame=<Frame name= url='https://aniagotuje.pl/przepis/murzynek-z-wisniami'> selector='span[class^="ingredient"] >> nth=0'>, 'amount': <Locator frame=<Frame name= url='https://aniagotuje.pl/przepis/murzynek-z-wisniami'> selector='span[class^="qty"] >> nth=0'>}, {'name': <Locator frame=<Frame name= url='https://aniagotuje.pl/przepis/murzynek-z-wisniami'> selector='span[class^="ingredient"] >> nth=1'>, 'amount': <Locator frame=<Frame name= url='https://aniagotuje.pl/przepis/murzynek-z-wisniami'> selector='span[class^="qty"] >> nth=1'>}, {'name': <Locator frame=<Frame name= url='https://aniagotuje.pl/przepis/murzynek-z-wisniami'> selector='span[class^="ingredient"] >> nth=2'>, 'amount': <Locator frame=<Frame name= url='https://aniagotuje.pl/przepis/murzynek-z-wisniami'> selector='span[class^="qty"] >> nth=2'>}, {'name': <Locator frame=<Frame name= url='http